## API access

In [301]:
from tweepy.auth import OAuthHandler, API, TweepError

In [302]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = API(auth)
print('Successfully connected to the Twitter API.')

Successfully connected to the Twitter API.


## Search Tweets

Now we can define our query and search for the tweets containing it.

- **query**: *hashtag* or *emoji* that will be used to fetch the tweets
- **max_requests**: Maximum number of requests to the API.
    - Restriction: 180 requests / 15 min window

In [303]:
query = ':face_with_steam_from_nose:'
max_request = 180

In [304]:
# Converts aliases to the real emoji representation (e.g. :thumbs_up: => 👍)

from emoji import emojize

In [305]:
help(emojize)

Help on function emojize in module emoji.core:

emojize(string, use_aliases=False, delimiters=(':', ':'))
    Replace emoji names in a string with unicode codes.
    
    :param string: String contains emoji names.
    :param use_aliases: (optional) Enable emoji aliases.  See ``emoji.UNICODE_EMOJI_ALIAS``.
    :param delimiters: (optional) Use delimiters other than _DEFAULT_DELIMITER
        >>> import emoji
        >>> print(emoji.emojize("Python is fun :thumbsup:", use_aliases=True))
        Python is fun 👍
        >>> print(emoji.emojize("Python is fun :thumbs_up:"))
        Python is fun 👍
        >>> print(emoji.emojize("Python is fun __thumbs_up__", delimiters = ("__", "__")))
        Python is fun 👍



In [306]:
emojize(':thumbs_up:')

'👍'

In [307]:
emojize(':fearful_face:')

'😨'

In [308]:
q = emojize(query) + ' -filter:retweets'
searched_tweets = []
last_id = -1 
request_count = 0

while request_count < max_request:
    try: 
        new_tweets = api.search(q=q,
                                lang='en',
                                count=100,
                                max_id=str(last_id - 1),
                                tweet_mode ='extended')
        if not new_tweets:
            break
        searched_tweets.extend(new_tweets)
        last_id = new_tweets[-1].id
        request_count += 1
    except TweepError as e:
        print(e)
        break

## Format and save

Formatting the API data to the desired structure and save a `.csv` file

In [309]:
import pandas as pd

In [310]:
data = []
for tweet in searched_tweets:
    data.append([tweet.id, tweet.created_at, tweet.user.screen_name, tweet.full_text])
df = pd.DataFrame(data, columns=['id', 'date', 'user', 'text'])
print(str(len(data)) + ' ' + query + 'tweets')

18000 :face_with_steam_from_nose:tweets


In [311]:
df.head()

,id,date,user,text
0,1242294442049986561,2020-03-24 03:37:30,edenvill,At this times of health crisis! WTF!😖😤 https:/...
1,1242294437293637632,2020-03-24 03:37:29,Camila__0511,Props to me for handling my online classes as ...
2,1242294436782059520,2020-03-24 03:37:29,MissPam_87,@8SandyCheeks Lmao hell nah! 😤
3,1242294436517871626,2020-03-24 03:37:29,cosmicc_humann,GOD THE MATENROU DRAMA TRACK IS REALLY GOING T...
4,1242294434550517761,2020-03-24 03:37:28,benadryllergy,so this is what theyve been keeping from us hu...


In [312]:
len(df)

18000

In [313]:
from pathlib import Path
import os

In [314]:
PATH = Path('C:\\Users\\Araceli\\Documents\\Git\\SentimentAnalysisTwitter\\datasets\\tweepy').resolve()
filename = 'face_with_steam_from_nose' + '.csv'
df.to_csv(os.path.join(PATH, filename), index = None)

print('Save under: "' + str(PATH) + '"')

Save under: "C:\Users\Araceli\Documents\Git\SentimentAnalysisTwitter\datasets\tweepy"
